# Evaluación de un TFG

En este Notebook, se implementa la evaluación de un documento de TFG en base a una rúbrica dada. En este caso, no se hace uso de RAG, dado que se va a usar la API de OpenAI y se permite un contexto de más de 100k tokens. 

Por lo tanto, en primer lugar, se usa la API de **Document Intelligence** para extraer el texto de dos documentos, el TFG y la rúbrica.

En segundo lugar, se diseña un *prompt* de ChatGPT que permite evaluar el documento en base a la rúbrica. La llamada a los servicios de OpenAI se realiza a través de un SDK Python.

# Instalación de librerías

In [ ]:
!pip install openai
!pip install azure-ai-formrecognizer
!pip install azure-ai-textanalytics
!pip install azure-ai-vision-imageanalysis

# Importación de librerías

In [ ]:
import os
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

In [ ]:
load_dotenv()

# Uso de DocumentIntelligence para extraer texto de PDF's.

In [0]:
# Obtener las variables de entorno con los endpoints y claves.
endpoint = os.getenv("AZURE_SERVICES_ENDPOINT")
key = os.getenv("AZURE_SERVICES_KEY")

# (Primero tendremos que subir los PDF al Catalog)
path_tfg = "/Volumes/uado01in/raw/uado01in/TFG_LUPION LORENTE, MARCOS.pdf"  
path_rubrica = "/Volumes/uado01in/raw/uado01in/AnexoIV-RubricaEvaluacionTFG.pdf"

In [0]:
# Create the client
document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# Analyse the file
with open(path_tfg, "rb") as f:
  poller = document_analysis_client.begin_analyze_document("prebuilt-layout",document=f).result()

text_tfg = []
for res in poller.to_dict()["paragraphs"]:
    if "content" in res:
        text_tfg.append(res["content"])

# Analyse the file
with open(path_rubrica, "rb") as f:
  poller = document_analysis_client.begin_analyze_document("prebuilt-layout",document=f).result()

text_word= []
for res in poller.to_dict()["paragraphs"]:
    if "content" in res:
        text_word.append(res["content"])

# Uso de OpenAI para evaluar el TFG

In [0]:
# Obtener las variables de entorno con los endpoints y claves.
openai.api_type = os.getenv("AZURE_OPENAI_API_TYPE")
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")

message_text = [{"role":"system","content":"Eres un evaluador de un trabajo final de grado en Ingeniería Informática. "},
    {"role":"user","content": f"Quiero que evalúe el TFG proporcionado en el contexto siguiente, 
    haciendo uso de la siguiente rúbrica. Además, ve siguiendo punto por punto los diferentes apartados de la rubrica 
    (calidad del trabajo, calidad de la memoria y calidad de la presentacicón, con sus competencias uy subapartados. 
    Evaluar cada uno de ellos mostrando ejemplos de errores que hacen que no se tengan la máxima puntuación en ellos. 
    Poner una nota entre 1 y 5 en cada subapartado. Sumar todas las notas de estos y dividirlas entre el máximo total 
    de cada subapartado. \n Contexto: {text_tfg}, Rúbrica: {text_word}"}] 

# Creamos una instancia de cliente de AzureOpenAI
client = AzureOpenAI(
  azure_endpoint = "", # El endpoint a usar lo daremos al inicio de la competición
  api_key="",  # La clave también os la daremos al comenzar la competición
  api_version=""
)

completion = client.chat.completions.create(
  model="gpt-4-turbo", # IMPORTANTE: aquí hay que poner el nombre del DEPLOYMENT a usar: model = "deployment_name"
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

print(completion)
print(completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-AHbab71wwzg3JyUqDxiuctjfK7mn2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='### Evaluación del Trabajo Fin de Grado - Marcos Lupión Lorente\n\n#### Calidad del Trabajo Fin de Grado\n**Justificación:**\nEl estudiante ha logrado un desarrollo técnico adecuado en la implementación del sistema IoT para una vivienda inteligente, utilizando tecnologías y herramientas modernas como MongoDB, Express, React, NodeJS, y Z-Wave. Sin embargo, no se detalla la integración y prueba de estos sistemas en un entorno real, lo cual es crucial para validar la funcionalidad y robustez del sistema en situaciones prácticas.\n\n- Identifica y relaciona ideas: 5\n- Resolución de problemas: 4.5 (No se mencionan pruebas en un entorno real)\n- Calidad y complejidad del trabajo: 5\n- Justificación del trabajo: 4.5 (Se podría mejorar explicando la elección de tecnologías específicas)\n- Desarrollo coherente con la metodología: 4.5 (

In [0]:
from IPython.display import display, Markdown
display(Markdown(completion.choices[0].message.content))

### Evaluación del Trabajo Fin de Grado - Marcos Lupión Lorente

#### Calidad del Trabajo Fin de Grado
**Justificación:**
El estudiante ha logrado un desarrollo técnico adecuado en la implementación del sistema IoT para una vivienda inteligente, utilizando tecnologías y herramientas modernas como MongoDB, Express, React, NodeJS, y Z-Wave. Sin embargo, no se detalla la integración y prueba de estos sistemas en un entorno real, lo cual es crucial para validar la funcionalidad y robustez del sistema en situaciones prácticas.

- Identifica y relaciona ideas: 5
- Resolución de problemas: 4.5 (No se mencionan pruebas en un entorno real)
- Calidad y complejidad del trabajo: 5
- Justificación del trabajo: 4.5 (Se podría mejorar explicando la elección de tecnologías específicas)
- Desarrollo coherente con la metodología: 4.5 (Falta de detalles en las pruebas)
- Uso de recursos específicos: 5
- Aportaciones ordenadas: 4.5 (Falta claridad en la integración de subsistemas)
- Otras consideraciones: 5

**Calificación**: 4.6

#### Calidad de la Memoria
**Justificación:**
La memoria está bien estructurada y cumple con los estándares académicos requeridos. La redacción es clara, aunque se podrían mejorar algunos aspectos de la sintaxis y la coherencia en la explicación de los procesos técnicos. Las referencias están bien citadas y el documento incluye todos los elementos gráficos necesarios para una comprensión adecuada del trabajo.

- Estilo adecuado: 5
- Redacción clara: 4.5
- Índice y numeración correcta: 5
- Presentación de fases y cronograma: 5
- Calidad de tablas y figuras: 5
- Referencias completas: 5
- Otras consideraciones: 5

**Calificación**: 4.9

#### Calidad de la Presentación
**Justificación:**
La presentación fue estructurada y cumplió con los tiempos establecidos. Sin embargo, se observa que podría mejorar en cuanto a la interacción con la audiencia y el uso de recursos comunicativos para hacer la presentación más dinámica y atractiva.

- Exposición ordenada: 5
- Solidez y rigor: 4.5
- Mantenimiento de atención: 4.5
- Respuestas a comentarios: 5
- Corrección y fluidez: 5
- Otras consideraciones: 5

**Calificación**: 4.8

#### Calificación Final
**Suma de Indicadores**: (4.6 * 5) + (4.9 * 2.5) + (4.8 * 2.5) = 23 + 12.25 + 12 = **47.25 / 10 = 4.725**

Redondeado a dos decimales, la calificación final es **4.73 / 5.0**, que convertido a la escala de 10, resulta en un **9.46 / 10**.

El trabajo demuestra un buen nivel técnico y académico, aunque se recomienda mejorar en las áreas de prueba y validación del sistema, así como en las técnicas de presentación para futuros trabajos o desarrollos profesionales.